In [1]:
!pip install -U scikit-learn

In [2]:
!apt -qq install -y fonts-nanum

The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Selecting previously unselected package fonts-nanum.
(Reading database ... 126333 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [22]:
!pip install streamlit pyngrok duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [24]:
# ✅ 1. 라이브러리
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager as fm
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBRegressor
import time

sns.set(style="whitegrid")

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

print("🚀 글로벌 모델 학습 및 검증 프로세스 시작!")

🚀 글로벌 모델 학습 및 검증 프로세스 시작!


In [25]:
# ✅ CSV 파일 경로 (Google Drive)
file_path = '/content/drive/MyDrive/company_analysis_sample_10000.csv'

# ✅ 데이터 로드
print("📊 CSV 파일에서 데이터 로딩 중...")
company_df = pd.read_csv(file_path, encoding='utf-8-sig')
print(f"✅ 데이터 로드 완료! 총 {len(company_df)} 행")

# ✅ DuckDB 연결
con = duckdb.connect(database='company_data.duckdb', read_only=False)

# ✅ DuckDB 저장
print("📁 DuckDB 에 데이터 저장 중...")
con.execute("CREATE OR REPLACE TABLE company_data AS SELECT * FROM company_df")
print("✅ DuckDB 저장 완료!")
con.close()

📊 CSV 파일에서 데이터 로딩 중...
✅ 데이터 로드 완료! 총 44771 행
📁 DuckDB 에 데이터 저장 중...
✅ DuckDB 저장 완료!


In [26]:
# ✅ 2. 데이터 로딩
print("📥 DuckDB 에서 데이터 로딩 중...")
con = duckdb.connect(database='company_data.duckdb', read_only=False)
df = con.execute("SELECT * FROM company_data").df()
con.close()
print(f"✅ 데이터 로딩 완료! 총 {len(df)} 행")

📥 DuckDB 에서 데이터 로딩 중...
✅ 데이터 로딩 완료! 총 44771 행


In [27]:
# ✅ 3. 피처 엔지니어링
df = df.sort_values(by=['사업자번호', '기준연도'])
group = df.groupby('사업자번호')

# 전년도, 2년 전 매 피쳐
df['전년_매출액'] = group['손익계산서_매출액'].shift(1)
df['2년전_매출액'] = group['손익계산서_매출액'].shift(2)
df['전년_자산총계'] = group['재무상태표_자산총계'].shift(1)
df['2년전_자산총계'] = group['재무상태표_자산총계'].shift(2)
df['전년_부채총계'] = group['재무상태표_부채총계'].shift(1)
df['2년전_부채총계'] = group['재무상태표_부채총계'].shift(2)
df['전년_근로자수'] = group['근로자수'].shift(1)
df['2년전_근로자수'] = group['근로자수'].shift(2)

# 증가율 피처
df['자산증가율'] = (df['전년_자산총계'] - df['2년전_자산총계']) / df['2년전_자산총계'].replace(0, np.nan)
df['부채증가율'] = (df['전년_부채총계'] - df['2년전_부채총계']) / df['2년전_부채총계'].replace(0, np.nan)
df['근로자수증가율'] = (df['전년_근로자수'] - df['2년전_근로자수']) / df['2년전_근로자수'].replace(0, np.nan)

# 기존 피처
df['전년_영업이익'] = group['손익계산서_영업이익'].shift(1)
df['전년_매출증가율'] = np.where(df['2년전_매출액'] > 0,
                          (df['전년_매출액'] - df['2년전_매출액']) / df['2년전_매출액'], np.nan)
df['전년_영업이익률'] = np.where(df['전년_매출액'] > 0,
                          df['전년_영업이익'] / df['전년_매출액'], np.nan)

# 업종 평균 대비
industry_avg = df.groupby('업종')[['손익계산서_매출액', '손익계산서_영업이익']].transform('mean')
df['업종평균_매출차이'] = df['손익계산서_매출액'] - industry_avg['손익계산서_매출액']
df['업종평균_이익차이'] = df['손익계산서_영업이익'] - industry_avg['손익계산서_영업이익']

# 안전 로그 변환
def safe_log(x):
   return np.sign(x) * np.log1p(np.abs(x))

log_cols = ['손익계산서_매출액', '손익계산서_영업이익', '재무상태표_자산총계',
         '재무상태표_부채총계', '재무상태표_자본총계',
         '재무상태표_유동자산', '재무상태표_유동부채']

for col in log_cols:
   df[f'safe_log_{col}'] = safe_log(df[col])

df['순입사자수'] = df['입사자수'].fillna(0) - df['퇴사자수'].fillna(0)
df['입사율'] = df['입사자수'].fillna(0) / df['근로자수'].replace(0, np.nan)
df['퇴사율'] = df['퇴사자수'].fillna(0) / df['근로자수'].replace(0, np.nan)

df['유동비율'] = df['재무상태표_유동자산'] / df['재무상태표_유동부채'].replace(0, np.nan)
df['부채비율'] = df['재무상태표_부채총계'] / df['재무상태표_자본총계'].replace(0, np.nan)
df['자본잉여금비율'] = df['재무상태표_자본잉여금'] / df['재무상태표_자본금'].replace(0, np.nan)
df['safe_log_유동비율'] = safe_log(df['유동비율'])
df['safe_log_부채비율'] = safe_log(df['부채비율'])

df['업종코드'] = LabelEncoder().fit_transform(df['업종'].astype(str))

print("✅ 피처 엔지니어링 완료!")

✅ 피처 엔지니어링 완료!


In [28]:
# ✅ 4. Train/Test Split
train_df = df[df['기준연도'] <= df['기준연도'].max() - 1].copy()
test_df = df[df['기준연도'] == df['기준연도'].max()].copy()

for dataset in [train_df, test_df]:
   dataset.loc[:, '매출증가율'] = np.where(
       dataset['전년_매출액'] > 0,
       (dataset['손익계산서_매출액'] - dataset['전년_매출액']) / dataset['전년_매출액'],
       np.nan
   )
   dataset.loc[:, '영업이익률'] = np.where(
       dataset['손익계산서_매출액'] > 0,
       dataset['손익계산서_영업이익'] / dataset['손익계산서_매출액'],
       np.nan
   )

features = [
   '전년_매출액', '전년_영업이익', '전년_매출증가율', '전년_영업이익률',
   '자산증가율', '부채증가율', '근로자수증가율',
   'safe_log_손익계산서_매출액', 'safe_log_손익계산서_영업이익',
   'safe_log_재무상태표_자산총계', 'safe_log_재무상태표_부채총계',
   'safe_log_재무상태표_자본총계', '근로자수', '순입사자수', '입사율', '퇴사율',
   '업종평균_매출차이', '업종평균_이익차이',
   'safe_log_유동비율', 'safe_log_부채비율', '자본잉여금비율',
   '업종코드'
]

target_growth = '매출증가율'
target_profit = '영업이익률'

# ✅ 타겟만 dropna
train_df = train_df.dropna(subset=[target_growth, target_profit])
test_df = test_df.dropna(subset=[target_growth, target_profit])

print(f"✅ 학습용 데이터: {len(train_df)} rows")
print(f"✅ 검증용 데이터: {len(test_df)} rows")

# ✅ 피처는 NaN 대체
X_train = train_df[features].fillna(0)
y_growth_train = train_df[target_growth]
y_profit_train = train_df[target_profit]

X_test = test_df[features].fillna(0)
y_growth_test = test_df[target_growth]
y_profit_test = test_df[target_profit]

# ✅ 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

✅ 학습용 데이터: 31434 rows
✅ 검증용 데이터: 453 rows


In [29]:
# # ✅ 5. 모델 학습 및 검증
# models = {
#     "HistGradientBoosting": HistGradientBoostingRegressor(random_state=42),
#     "XGBoost": XGBRegressor(random_state=42, n_estimators=100, verbosity=0),
#     "RandomForest": RandomForestRegressor(random_state=42, n_estimators=100),
# }

# def evaluate_model(model, name, X_train, X_test, y_train, y_test, target_name):
#     start = time.time()
#     model.fit(X_train, y_train)
#     end = time.time()
#     y_pred = model.predict(X_test)

#     rmse = np.sqrt(mean_squared_error(y_test * 100, y_pred * 100))
#     mae = mean_absolute_error(y_test * 100, y_pred * 100)

#     print(f"\n🎯 {name} - {target_name} 학습 완료! (시간: {end - start:.2f}초)")
#     print(f"➡️ RMSE: {rmse:.2f}%, MAE: {mae:.2f}%")

# # 각각 모델별 평가
# for model_name, model_instance in models.items():
#     print(f"\n🚀 모델 평가: {model_name}")
#     evaluate_model(model_instance, model_name, X_train_scaled, X_test_scaled, y_growth_train, y_growth_test, "매출증가율")
#     evaluate_model(model_instance, model_name, X_train_scaled, X_test_scaled, y_profit_train, y_profit_test, "영업이익률")

# print("\n🎉 모든 모델 학습 및 검증 완료!")


🚀 모델 평가: HistGradientBoosting

🎯 HistGradientBoosting - 매출증가율 학습 완료! (시간: 0.30초)
➡️ RMSE: 7183.16%, MAE: 654.60%

🎯 HistGradientBoosting - 영업이익률 학습 완료! (시간: 0.35초)
➡️ RMSE: 666.85%, MAE: 96.82%

🚀 모델 평가: XGBoost

🎯 XGBoost - 매출증가율 학습 완료! (시간: 0.91초)
➡️ RMSE: 129.81%, MAE: 20.87%

🎯 XGBoost - 영업이익률 학습 완료! (시간: 0.67초)
➡️ RMSE: 123.19%, MAE: 14.40%

🚀 모델 평가: RandomForest


KeyboardInterrupt: 

In [30]:
# ✅ 5. RandomForest 기반 모델 학습
model_growth = RandomForestRegressor(random_state=42, n_estimators=100)
print("🔴 RandomForest 모델 학습 시작 (매출증가율)...")
model_growth.fit(X_train_scaled, y_growth_train)
print("✅ 매출증가율 모델 학습 완료!")

model_profit = RandomForestRegressor(random_state=42, n_estimators=100)
print("🔴 RandomForest 모델 학습 시작 (영업이익률)...")
model_profit.fit(X_train_scaled, y_profit_train)
print("✅ 영업이익률 모델 학습 완료!")

# ✅ 6. 검증
y_growth_pred = model_growth.predict(X_test_scaled)
y_profit_pred = model_profit.predict(X_test_scaled)

rmse_growth = np.sqrt(mean_squared_error(y_growth_test * 100, y_growth_pred * 100))
mae_growth = mean_absolute_error(y_growth_test * 100, y_growth_pred * 100)

rmse_profit = np.sqrt(mean_squared_error(y_profit_test * 100, y_profit_pred * 100))
mae_profit = mean_absolute_error(y_profit_test * 100, y_profit_pred * 100)

print(f"📊 매출증가율 RMSE: {rmse_growth:.2f}%, MAE: {mae_growth:.2f}%")
print(f"📊 영업이익률 RMSE: {rmse_profit:.2f}%, MAE: {mae_profit:.2f}%")

🔴 RandomForest 모델 학습 시작 (매출증가율)...
✅ 매출증가율 모델 학습 완료!
🔴 RandomForest 모델 학습 시작 (영업이익률)...
✅ 영업이익률 모델 학습 완료!
📊 매출증가율 RMSE: 54.36%, MAE: 9.52%
📊 영업이익률 RMSE: 77.64%, MAE: 8.24%


In [31]:

# ✅ 7. 미래 예측 및 저장
def predict_future(model_growth, model_profit, scaler, original_df, years_ahead=3):
    prediction_results = []
    print("🔮 미래 예측 진행 중...")
    biz_total = train_df['사업자번호'].nunique()
    for idx, (biz_no, group) in enumerate(train_df.groupby('사업자번호'), start=1):
        if idx % 100 == 0:
            print(f"➡️ {idx}/{biz_total} 기업 예측 완료")
        group = group.sort_values('기준연도')
        if len(group) < 2:
            continue

        last_row = group.iloc[-1].copy()

        for i in range(1, years_ahead + 1):
            next_year = last_row['기준연도'] + 1
            input_features = pd.DataFrame([{feat: last_row.get(feat, 0) for feat in features}])
            input_scaled = scaler.transform(input_features)

            predicted_growth = model_growth.predict(input_scaled)[0]
            predicted_profit = model_profit.predict(input_scaled)[0]

            prediction_results.append({
                '사업자번호': biz_no,
                '기업명': last_row['기업명'],
                '기준연도': next_year,
                '예측_매출증가율': predicted_growth * 100,
                '예측_영업이익률': predicted_profit * 100
            })

            last_row['기준연도'] = next_year
            last_row['손익계산서_매출액'] *= (1 + predicted_growth)
            last_row['매출증가율'] = predicted_growth
            last_row['영업이익률'] = predicted_profit

    prediction_df = pd.DataFrame(prediction_results)
    print(f"✅ 미래 예측 완료! 총 {len(prediction_df)}건")
    return prediction_df

future_predictions = predict_future(model_growth, model_profit, scaler, train_df)

if not future_predictions.empty:
    con = duckdb.connect(database='company_data.duckdb', read_only=False)
    con.execute("DROP TABLE IF EXISTS prediction_results")
    con.execute("CREATE TABLE prediction_results AS SELECT * FROM future_predictions")
    con.close()
    print("✅ 예측 결과 DuckDB 저장 완료!")
else:
    print("⚠️ 예측 결과가 비어 있습니다. 저장을 건너뜁니다.")

print("🎉 전체 프로세스 완료! Streamlit 대시보드에서 바로 확인하세요.")

🔮 미래 예측 진행 중...
➡️ 100/8914 기업 예측 완료
➡️ 200/8914 기업 예측 완료
➡️ 300/8914 기업 예측 완료
➡️ 400/8914 기업 예측 완료
➡️ 500/8914 기업 예측 완료
➡️ 600/8914 기업 예측 완료
➡️ 700/8914 기업 예측 완료
➡️ 800/8914 기업 예측 완료
➡️ 900/8914 기업 예측 완료
➡️ 1000/8914 기업 예측 완료
➡️ 1100/8914 기업 예측 완료
➡️ 1200/8914 기업 예측 완료
➡️ 1300/8914 기업 예측 완료
➡️ 1400/8914 기업 예측 완료
➡️ 1500/8914 기업 예측 완료
➡️ 1600/8914 기업 예측 완료
➡️ 1700/8914 기업 예측 완료
➡️ 1800/8914 기업 예측 완료
➡️ 1900/8914 기업 예측 완료
➡️ 2000/8914 기업 예측 완료
➡️ 2100/8914 기업 예측 완료
➡️ 2200/8914 기업 예측 완료
➡️ 2300/8914 기업 예측 완료
➡️ 2400/8914 기업 예측 완료
➡️ 2500/8914 기업 예측 완료
➡️ 2600/8914 기업 예측 완료
➡️ 2700/8914 기업 예측 완료
➡️ 2800/8914 기업 예측 완료
➡️ 2900/8914 기업 예측 완료
➡️ 3000/8914 기업 예측 완료
➡️ 3100/8914 기업 예측 완료
➡️ 3200/8914 기업 예측 완료
➡️ 3300/8914 기업 예측 완료
➡️ 3400/8914 기업 예측 완료
➡️ 3500/8914 기업 예측 완료
➡️ 3600/8914 기업 예측 완료
➡️ 3700/8914 기업 예측 완료
➡️ 3800/8914 기업 예측 완료
➡️ 3900/8914 기업 예측 완료
➡️ 4000/8914 기업 예측 완료
➡️ 4100/8914 기업 예측 완료
➡️ 4200/8914 기업 예측 완료
➡️ 4300/8914 기업 예측 완료
➡️ 4400/8914 기업 예측 완료
➡️ 4500/8914 기업 예측 완료
➡️ 

In [32]:
# ngrok으로 공개 URL 생성 (Authtoken 필요)
from pyngrok import ngrok

# 여기에 발급받은 ngrok authtoken 입력하세요!
NGROK_AUTH_TOKEN = "2w1lMu0vDS1pgY2wDQ0d4mJ8IyP_2Jim9cP1bo2r65A59DGmo"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("[INFO] ngrok 터널 생성 중...")
public_url = ngrok.connect(addr="8501", proto="http")
print(f"[INFO] Streamlit 앱 Public URL: {public_url}")

# Streamlit 앱 실행
!streamlit run app2.py --server.port 8501 --server.enableCORS false

print("[INFO] Streamlit 웹 서비스가 실행되었습니다. 위의 Public URL을 클릭하여 접속하세요!")

[INFO] ngrok 터널 생성 중...
[INFO] Streamlit 앱 Public URL: NgrokTunnel: "https://e492-34-124-155-55.ngrok-free.app" -> "http://localhost:8501"
2025-04-23 01:39:31.675 
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            
Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app2.py
[INFO] Streamlit 웹 서비스가 실행되었습니다. 위의 Public URL을 클릭하여 접속하세요!


In [ ]:
from sklearn.model_selection import GridSearchCV

# ✅ 13. RandomForestRegressor (Grid Search)
print("\n🔍 RandomForest + Grid Search 시작!")

param_grid = {
   'n_estimators': [100, 200],
   'max_depth': [5, 10, None],
   'min_samples_split': [2, 5],
}

def grid_search_and_evaluate(X_train, X_test, y_train, y_test, target_name):
   print(f"🔎 Grid Search - {target_name} 시작!")
   grid_search = GridSearchCV(
       RandomForestRegressor(random_state=42),
       param_grid,
       cv=3,
       scoring='neg_mean_squared_error',
       verbose=2
   )
   grid_search.fit(X_train, y_train)

   best_model = grid_search.best_estimator_
   print(f"✅ [GridSearch] {target_name} Best Params: {grid_search.best_params_}")

   predictions = best_model.predict(X_test)
   rmse = np.sqrt(mean_squared_error(y_test, predictions))
   mae = mean_absolute_error(y_test, predictions)

   print(f"📊 [GridSearch] {target_name} RMSE: {rmse:.2f}, MAE: {mae:.2f}")
   return best_model

# ✅ Grid Search - 매출증가율
best_growth_model = grid_search_and_evaluate(X_train, X_test, y_growth_train, y_growth_test, "매출증가율")

# ✅ Grid Search - 영업이익률
best_profit_model = grid_search_and_evaluate(X_train, X_test, y_profit_train, y_profit_test, "영업이익률")

print("\n✅ 모든 모델 학습 및 평가 완료!")